# Project 05 — Quant Strategy Backtest (Momentum vs Mean Reversion)

**Goal:** implement two classic signals, run a clean backtest, and report professional metrics.

You will:
- Build signals (momentum / mean reversion)
- Generate positions (long-only or long/short)
- Compute PnL + equity curve with transaction costs
- Compute Sharpe, max drawdown, turnover, cost drag
- Use interactive controls to test sensitivity


## Environment / requirements

Recommended Python packages:
- `numpy`, `pandas`
- `plotly`, `ipywidgets`
- `yfinance`

If you are missing a package, run:

```bash
pip install numpy pandas plotly ipywidgets yfinance
```


In [1]:

from __future__ import annotations
from pathlib import Path

SEED = 42

PROJECT_DIR = Path.cwd()
ASSETS_DIR = PROJECT_DIR / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", PROJECT_DIR)
print("ASSETS_DIR:", ASSETS_DIR.resolve())


CWD: c:\Users\Karim\Desktop\quant-finance-portfolio\projects\05_backtest_strategy
ASSETS_DIR: C:\Users\Karim\Desktop\quant-finance-portfolio\projects\05_backtest_strategy\assets


In [ ]:

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf

import ipywidgets as widgets
from IPython.display import display


def _ensure_series(prices, name: str = "price") -> pd.Series:
    """Coerce a price input (Series or 1-col DataFrame) to a float Series."""
    if isinstance(prices, pd.DataFrame):
        if prices.shape[1] == 1:
            s = prices.iloc[:, 0]
        else:
            # Try common column names, else take the first column
            preferred = ["Adj Close", "Close", "Price", "price", "adj_close", "close"]
            col = next((c for c in preferred if c in prices.columns), prices.columns[0])
            s = prices[col]
    elif isinstance(prices, pd.Series):
        s = prices
    else:
        s = pd.Series(prices)

    s = pd.to_numeric(s, errors="coerce").astype(float)
    s.name = name
    return s


## 1) Data

We use adjusted close prices and daily log-returns:

\[
r_t = \log(S_t) - \log(S_{t-1})
\]

**Interpretation:** positive return means price went up; log-returns are additive across time.


In [3]:

def download_prices(ticker: str = "SPY", start: str = "2010-01-01") -> pd.Series:
    s = yf.download(ticker, start=start, auto_adjust=True, progress=False)["Close"].dropna()
    s.name = "close"
    return s

def log_returns(prices: pd.Series) -> pd.Series:
    r = np.log(prices).diff().dropna()
    r.name = "r"
    return r

ticker = "SPY"
prices = download_prices(ticker, start="2010-01-01")
r = log_returns(prices)

print("prices:", prices.shape, "returns:", r.shape)


prices: (4016, 1) returns: (4015, 1)


In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=prices.index, y=prices.values, mode="lines", name=ticker))
fig.update_layout(template="plotly_dark", title=f"{ticker} — Adjusted Close", xaxis_title="Date", yaxis_title="Price")
fig.show()
fig.write_html(ASSETS_DIR / "price_series.html")

# Robust handling: r can be a Series (expected) or a DataFrame (if you changed the download/signal logic)
if isinstance(r, pd.DataFrame):
    if "r" in r.columns:
        r_series = r["r"]
    elif r.shape[1] == 1:
        r_series = r.iloc[:, 0]
    else:
        # Fallback: take the first column (you can change this to a specific ticker/column)
        r_series = r.iloc[:, 0]
    r_series = r_series.dropna()
    r_series.name = "r"
else:
    r_series = pd.Series(r).dropna()
    r_series.name = "r"

fig2 = px.histogram(
    r_series.to_frame(),
    x="r",
    nbins=120,
    template="plotly_dark",
    title="Daily log-return distribution",
)
fig2.show()
fig2.write_html(ASSETS_DIR / "return_hist.html")


### How to read these charts
- Price series: long-term drift upward (equities), but with large drawdowns.
- Return histogram: often heavy tails (more extreme moves than a Normal).


## 2) Strategy #1 — Time-series momentum

Simple momentum over a lookback window \(L\):

\[
\text{mom}_t = \frac{S_t}{S_{t-L}} - 1
\]

Position (long/short):
\[
p_t = \operatorname{sign}(\text{mom}_t)
\]

**Interpretation:**  
If the asset went up over the past \(L\) days, we go long; if it went down, we go short.


In [5]:

def momentum_positions(prices: pd.Series, lookback: int = 20, long_only: bool = True) -> pd.Series:
    prices = _ensure_series(prices, name="price")
    mom = prices / prices.shift(lookback) - 1.0
    pos = np.sign(mom).replace(0.0, np.nan).ffill().fillna(0.0)

    if long_only:
        pos = pos.clip(lower=0.0)

    pos.name = "pos"
    return pos

## 3) Strategy #2 — Mean reversion (z-score)

Compute a moving average and standard deviation:

\[
\text{MA}_t = \frac{1}{L}\sum_{i=0}^{L-1} S_{t-i}, \quad
\text{SD}_t = \sqrt{\frac{1}{L-1}\sum_{i=0}^{L-1} (S_{t-i}-\text{MA}_t)^2}
\]

Z-score:
\[
z_t = \frac{S_t - \text{MA}_t}{\text{SD}_t}
\]

Trading rule with hysteresis:
- enter short if \(z_t > z_{\text{entry}}\)
- enter long if \(z_t < -z_{\text{entry}}\)
- exit to flat if \(|z_t| < z_{\text{exit}}\)

**Interpretation:**  
When price is far above its average, we bet it will revert down (short), and vice-versa.


In [6]:

def mean_reversion_positions(
    prices: pd.Series,
    ma_window: int = 20,
    z_entry: float = 1.5,
    z_exit: float = 0.5,
    long_only: bool = False,
) -> pd.Series:
    prices = _ensure_series(prices, name="price")
    ma = prices.rolling(ma_window).mean()
    sd = prices.rolling(ma_window).std(ddof=1)
    z = (prices - ma) / sd

    pos = pd.Series(0.0, index=prices.index, name="pos")
    state = 0.0  # -1 short, 0 flat, +1 long

    for t in range(len(prices)):
        if pd.isna(z.iloc[t]):
            pos.iloc[t] = state
            continue

        if state == 0.0:
            if z.iloc[t] > z_entry:
                state = -1.0
            elif z.iloc[t] < -z_entry:
                state = +1.0
        else:
            if abs(z.iloc[t]) < z_exit:
                state = 0.0

        pos.iloc[t] = state

    if long_only:
        pos = pos.clip(lower=0.0)

    return pos

## 4) Backtest engine (with costs)

We apply a one-day shift to avoid look-ahead bias:
\[
\text{strategy return}_t = p_{t-1} \cdot r_t
\]

Turnover:
\[
\text{turnover}_t = |p_t - p_{t-1}|
\]

Costs (simple proportional model):
\[
\text{cost}_t = c \cdot \text{turnover}_t
\]


In [7]:

def backtest_single_asset(
    prices: pd.Series,
    pos: pd.Series,
    cost_bps: float = 1.0,
) -> dict:
    prices = _ensure_series(prices, name="price")
    pos = _ensure_series(pos, name="pos")
    # Align
    pos = pos.reindex(prices.index).fillna(0.0)
    r = np.log(prices).diff().fillna(0.0)

    pos_lag = pos.shift(1).fillna(0.0)

    strat_ret = pos_lag * r
    turnover = (pos - pos_lag).abs()
    costs = (cost_bps / 1e4) * turnover  # bps -> decimal

    net_ret = strat_ret - costs
    equity = (1.0 + net_ret).cumprod()

    out = {
        "ret": net_ret,
        "gross_ret": strat_ret,
        "equity": equity,
        "turnover": turnover,
        "costs": costs,
    }
    return out

## 5) Metrics (robust — fixes the “IndexError” issue)

The error you saw happens when `equity` becomes empty after alignment / `dropna()`.  
So the function below:
- realigns series safely,
- checks lengths before using `iloc[0]` / `iloc[-1]`,
- returns NaNs instead of crashing if the backtest has no data.


In [8]:

def max_drawdown(equity: pd.Series) -> float:
    if equity is None or len(equity) == 0:
        return np.nan
    eq = equity.dropna()
    if len(eq) == 0:
        return np.nan
    peak = eq.cummax()
    dd = eq / peak - 1.0
    return float(dd.min())

def perf_metrics(
    ret: pd.Series,
    equity: pd.Series | None = None,
    turnover: pd.Series | None = None,
    costs: pd.Series | None = None,
    periods_per_year: int = 252,
) -> dict:
    ret = ret.dropna()
    if len(ret) < 2:
        return {
            "ann_return": np.nan, "ann_vol": np.nan, "sharpe": np.nan,
            "max_dd": np.nan, "total_return": np.nan,
            "avg_turnover": np.nan, "total_costs": np.nan,
        }

    if equity is None:
        equity = (1.0 + ret).cumprod()
    equity = equity.reindex(ret.index).dropna()

    if len(equity) < 2:
        equity = (1.0 + ret).cumprod()

    total_return = float(equity.iloc[-1] / equity.iloc[0] - 1.0)
    ann_return = float((equity.iloc[-1] / equity.iloc[0]) ** (periods_per_year / max(1, len(ret))) - 1.0)

    ann_vol = float(ret.std(ddof=1) * np.sqrt(periods_per_year))
    sharpe = float((ret.mean() * periods_per_year) / (ret.std(ddof=1) * np.sqrt(periods_per_year) + 1e-12))

    mdd = max_drawdown(equity)

    avg_turnover = float(turnover.dropna().mean()) if turnover is not None else np.nan
    total_costs = float(costs.dropna().sum()) if costs is not None else np.nan

    return {
        "ann_return": ann_return,
        "ann_vol": ann_vol,
        "sharpe": sharpe,
        "max_dd": mdd,
        "total_return": total_return,
        "avg_turnover": avg_turnover,
        "total_costs": total_costs,
    }


## 6) Run both strategies (baseline)

Below we run:
- Momentum (long-only)
- Mean reversion (long/short)

Then we compare equity curves and metrics.


In [9]:

# Baseline parameters
lookback = 60
ma_window = 20
z_entry = 1.5
z_exit = 0.5
cost_bps = 1.0

pos_mom = momentum_positions(prices, lookback=lookback, long_only=True)
pos_mr = mean_reversion_positions(prices, ma_window=ma_window, z_entry=z_entry, z_exit=z_exit, long_only=False)

bt_mom = backtest_single_asset(prices, pos_mom, cost_bps=cost_bps)
bt_mr = backtest_single_asset(prices, pos_mr, cost_bps=cost_bps)

m_mom = perf_metrics(bt_mom["ret"], bt_mom["equity"], bt_mom["turnover"], bt_mom["costs"])
m_mr = perf_metrics(bt_mr["ret"], bt_mr["equity"], bt_mr["turnover"], bt_mr["costs"])

pd.DataFrame([m_mom, m_mr], index=["Momentum (LO)", "Mean reversion (LS)"]).round(4)


,ann_return,ann_vol,sharpe,max_dd,total_return,avg_turnover,total_costs
Momentum (LO),0.0827,0.1119,0.7665,-0.2109,2.5469,0.0401,0.0161
Mean reversion (LS),-0.0261,0.1377,-0.1234,-0.4827,-0.3444,0.1056,0.0424


In [10]:

eq_df = pd.DataFrame({
    "Momentum (LO)": bt_mom["equity"],
    "Mean reversion (LS)": bt_mr["equity"],
}).dropna()

fig = go.Figure()
for c in eq_df.columns:
    fig.add_trace(go.Scatter(x=eq_df.index, y=eq_df[c], mode="lines", name=c))
fig.update_layout(template="plotly_dark", title="Equity curves (net of costs)", xaxis_title="Date", yaxis_title="Equity")
fig.show()
fig.write_html(ASSETS_DIR / "equity_curves.html")


### Interpreting equity curves
- Look for **smoothness** (lower drawdowns) vs **growth** (higher returns).
- A strategy with high Sharpe but low return may be too conservative.
- Costs matter: high turnover strategies can look great “gross”, but weak “net”.


## 7) Interactive controls

Use sliders to explore:
- lookback for momentum
- MA window and thresholds for mean reversion
- transaction costs (bps)

**Tip:** If performance collapses when costs increase, the edge may be too fragile.


In [ ]:

ticker_lbl = widgets.Text(value=ticker, description="Ticker", layout=widgets.Layout(width="250px"))
lookback_w = widgets.IntSlider(value=60, min=5, max=252, step=5, description="Mom L", continuous_update=False)
ma_w = widgets.IntSlider(value=20, min=5, max=120, step=5, description="MR MA", continuous_update=False)
z_entry_w = widgets.FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description="z_entry", readout_format=".1f", continuous_update=False)
z_exit_w = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description="z_exit", readout_format=".1f", continuous_update=False)
cost_w = widgets.FloatSlider(value=1.0, min=0.0, max=20.0, step=0.5, description="cost bps", readout_format=".1f", continuous_update=False)

btn = widgets.Button(description="Run backtest", button_style="warning")

out = widgets.Output()

def run(_):
    with out:
        out.clear_output()

        tck = ticker_lbl.value.strip().upper()
        pr = download_prices(tck, start="2010-01-01")

        pos_mom = momentum_positions(pr, lookback=lookback_w.value, long_only=True)
        pos_mr = mean_reversion_positions(pr, ma_window=ma_w.value, z_entry=z_entry_w.value, z_exit=z_exit_w.value, long_only=False)

        bt_mom = backtest_single_asset(pr, pos_mom, cost_bps=cost_w.value)
        bt_mr = backtest_single_asset(pr, pos_mr, cost_bps=cost_w.value)

        m_mom = perf_metrics(bt_mom["ret"], bt_mom["equity"], bt_mom["turnover"], bt_mom["costs"])
        m_mr = perf_metrics(bt_mr["ret"], bt_mr["equity"], bt_mr["turnover"], bt_mr["costs"])

        display(pd.DataFrame([m_mom, m_mr], index=["Momentum (LO)", "Mean reversion (LS)"]).round(4))

        eq_df = pd.DataFrame({"Momentum (LO)": bt_mom["equity"], "Mean reversion (LS)": bt_mr["equity"]}).dropna()
        fig = go.Figure()
        for c in eq_df.columns:
            fig.add_trace(go.Scatter(x=eq_df.index, y=eq_df[c], mode="lines", name=c))
        fig.update_layout(template="plotly_dark", title=f"Equity curves — {tck} (net)", xaxis_title="Date", yaxis_title="Equity")
        fig.show()

btn.on_click(run)

display(widgets.HBox([ticker_lbl, btn]))
display(widgets.HBox([lookback_w, ma_w]))
display(widgets.HBox([z_entry_w, z_exit_w, cost_w]))
display(out)

# Auto-run once
run(None)


Output()

## 8) What to say in interviews (short)

- “I implemented two canonical signals (TSMOM and z-score mean reversion) with strict no-lookahead.”
- “I modeled transaction costs via turnover and reported net performance.”
- “I summarized results with Sharpe, max drawdown, and sensitivity via interactive controls.”
